In [1]:
import numpy as np
import os, sys
import neuroglancer
import matplotlib
from matplotlib import pyplot as plt

In [2]:
PATH = '/home/eddyod/programming/pipeline_utility'
sys.path.append(PATH)
from utilities.contour_utilities import get_structure_colors
VOL_DIR = '/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes/atlasV7/10.0um_annotationAsScoreVolume'
MD589_VOLUME_PATH = '/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes/MD589/MD589_full_filled.npy'
MD589 = np.load(MD589_VOLUME_PATH)
xy_ng_resolution_um = 5
neuroglancer.set_server_bind_address(bind_port='33645')

Connecting dklab@192.168.1.12:3306


In [ ]:
height, width, zdim = (1000,1000,288)
#full_brain_volume_annotated = np.zeros((zdim,height,width), dtype=np.uint8)
full_brain_volume_annotated = np.zeros((height, width, zdim), dtype=np.uint8)
files = os.listdir(VOL_DIR)
numpy_files = [f for f in files if f.endswith('.npy') and 'surround' not in f]
print('full brain volume shape:', full_brain_volume_annotated.shape)
midx = width // 2
midy = height // 2
midz = zdim // 2
print("origin", midx, midy, midz)
colors = get_structure_colors()

In [ ]:
full_brain_volume_annotated = np.zeros((height, width, zdim), dtype=np.uint8)
for n in numpy_files:
    structure = os.path.splitext(n)[0]
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    if structure not in ['SC', 'Sp5C_R']:
        continue
    surround_filename = os.path.join(VOL_DIR, f'{structure}_surround_200um.npy')
    surround_input = np.load(surround_filename)
    surround_input = np.swapaxes(surround_input, 0, 2)
    surround_nonzero_indices = surround_input > 0
    surround_input[surround_nonzero_indices] = 200
    surround_volume = surround_input.astype(np.uint8)

    volume_filename = os.path.join(VOL_DIR, f'{structure}.npy')
    volume_input = np.load(volume_filename)
    volume_input = np.swapaxes(volume_input, 0, 2)
    volume_nonzero_indices = volume_input > 0
    volume_input[volume_nonzero_indices] = color
    structure_volume = volume_input.astype(np.uint8)

    origin_filename = os.path.join(VOL_DIR, f'{structure}_origin_wrt_canonicalAtlasSpace.txt')
    origin_wrt = np.loadtxt(origin_filename)
    x,y,z = origin_wrt
    
    surround_origin_filename = os.path.join(VOL_DIR, f'{structure}_surround_200um_origin_wrt_canonicalAtlasSpace.txt')
    surround_origin_wrt = np.loadtxt(surround_origin_filename)
    sx,sy,sz = surround_origin_wrt
    

    z_start = 30
    
    y_start = int(y) + midy
    x_start = int(x) + midx
    x_end = x_start + structure_volume.shape[1]
    y_end = y_start + structure_volume.shape[0]
    z_end = z_start + structure_volume.shape[2]

    sy_start = int(sy) + midy
    sx_start = int(sx) + midx
    sx_end = sx_start + surround_volume.shape[1]
    sy_end = sy_start + surround_volume.shape[0]
    sz_end = z_start + surround_volume.shape[2]

    
    try:
        #full_brain_volume_annotated[z_start:z_end, y_start:y_end,x_start:x_end] = structure_volume
        full_brain_volume_annotated[y_start:y_end, x_start:x_end,z_start:z_end] = structure_volume
    except:
        print('Error',str(structure).ljust(8), str(color).rjust(2), end="\t")
        print('shape', str(structure_volume.shape).rjust(18), end=", ")
        print('x range', str(x_start).rjust(4), str(x_end).rjust(4), end=", ")
        print('y range', str(y_start).rjust(4), str(y_end).rjust(4), end=", ")
        print('z range', str(z_start).rjust(4), str(z_end).rjust(4))
    
    try:
        #full_brain_volume_annotated[z_start:z_end, y_start:y_end,x_start:x_end] = structure_volume
        full_brain_volume_annotated[sy_start:sy_end, sx_start:sx_end,z_start:sz_end] = surround_volume
    except:
        print('SError',str(structure).ljust(8), str(color).rjust(2), end="\t")
        print('shape', str(surround_volume.shape).rjust(18), end=", ")
        print('x range', str(sx_start).rjust(4), str(sx_end).rjust(4), end=", ")
        print('y range', str(sy_start).rjust(4), str(sy_end).rjust(4), end=", ")
        print('z range', str(z_start).rjust(4), str(sz_end).rjust(4))



In [ ]:
full_brain_volume_annotated.shape, full_brain_volume_annotated.dtype

In [ ]:
arr = full_brain_volume_annotated
limit = 10
start = (arr.shape[2] // 2) - limit
end = start + limit * 2
start = 70
end = 170
nrows = end - start
ncols = 1
plt.style.use('classic')
for i in range(30, 90, 10):
    plt.title('Z {}'.format(i))
    plt.imshow(arr[:,:,i], cmap='gray')
    #plt.imshow(mat, cmap='tab20', interpolation='none')
    plt.show()

In [ ]:
zdim = full_brain_volume_annotated.shape[2]
dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='um',
    scales=[5,5,10])
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.dimensions = dimensions
    s.layout = '4panel'
    s.layers['Altas'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(
            data=full_brain_volume_annotated,
            dimensions=dimensions,
            voxel_offset=(0, 0, 0),
        ),
        shader="""
#uicontrol float min slider(min=0, max=1, default=0)
#uicontrol float max slider(min=0, max=1, default=1)
#uicontrol float invert slider(min=0, max=1, default=0, step=1)
#uicontrol float brightness slider(min=-1, max=1)
#uicontrol float contrast slider(min=-3, max=3, step=0.01)
void main() {
  float pix_val = float(toRaw(getDataValue()));
  if(pix_val < min){
  	pix_val = 0.0;
  }
  if(pix_val > max){
    pix_val = 1.0;
  }

  if(invert==1.0){
  	  emitGrayscale((1.0 -(pix_val - brightness)) *
       exp(contrast));
  }
  else{
    emitGrayscale((pix_val + brightness) *
                  exp(contrast));
  }

}

""")

print(viewer)

In [ ]:
#neuroglancer.stop()